In [1]:
import pandas as pd
import torch as tc
import numpy as np
import multiprocessing
import re
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
# from pyevmasm import disassemble_hex
from web3 import Web3
from evmdasm import EvmBytecode
from datasets import *
from transformers import DataCollatorWithPadding
from torch import nn





/home/venglov/.venv/main/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# benign_df = pd.read_csv('../data/normal_df.csv').sample(frac=1).reset_index(drop=True)[:20000]
# malicious_df = pd.read_csv('../data/malicious.csv')
# sc_df = pd.concat([benign_df, malicious_df]).reset_index(drop=True)

In [3]:
# sc_df = pd.read_csv('/home/venglov/Documents/mcd_data/data_v2/opcodes_full_10k.csv')

In [4]:
# sc_df

In [5]:
# sc_df = sc_df[:10000]

In [6]:
# sc_df = sc_df[['decompiled_opcodes', 'malicious']].sample(frac=1).reset_index(drop=True)

In [7]:
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error"].index)
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error ;"].index)
# sc_df = sc_df.reset_index()
# sc_df

In [8]:

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
raw_dataset = load_dataset("csv", data_files="/home/venglov/Documents/mcd_data/data_v2/opcodes_full_10k.csv", split='train')
real_contracts = load_dataset("csv", data_files="/home/venglov/Documents/mcd_data/data_v2/contracts.csv", split='train')
raw_dataset = concatenate_datasets([raw_dataset, real_contracts])
new_features = raw_dataset.features.copy()
new_features["malicious"] = ClassLabel(names=["n", "p"])
raw_dataset = raw_dataset.cast(new_features)
# raw_dataset.features = {'label': ClassLabel(num_classes=2, names=['neg', 'pos'], id=None)}
raw_dataset = raw_dataset.train_test_split(0.1, 0.9, seed=42)

Found cached dataset csv (/home/venglov/.cache/huggingface/datasets/csv/default-2b849a0c54b184cb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/venglov/.cache/huggingface/datasets/csv/default-49e15014392d381f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/venglov/.cache/huggingface/datasets/csv/default-2b849a0c54b184cb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-03e3437ba700582d.arrow
Loading cached split indices for dataset at /home/venglov/.cache/huggingface/datasets/csv/default-2b849a0c54b184cb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-3004389aa81f0ab8.arrow and /home/venglov/.cache/huggingface/datasets/csv/default-2b849a0c54b184cb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-8e69061db0518ba4.arrow


In [9]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'contract_address', 'opcodes', 'malicious'],
        num_rows: 9257
    })
    test: Dataset({
        features: ['Unnamed: 0', 'contract_address', 'opcodes', 'malicious'],
        num_rows: 1029
    })
})

In [10]:


raw_dataset = raw_dataset.remove_columns(['Unnamed: 0', 'contract_address'])
raw_dataset = raw_dataset.rename_column('malicious', 'labels')



# raw_dataset = raw_dataset.map(lambda x: {'labels': 0 if x['labels'] else 1})
# raw_dataset = raw_dataset.map(lambda x: {'labels': ([0, 1] if x['labels'] else [1, 0])})

In [11]:
def tokenize_function(example):
    return tokenizer(example["opcodes"], truncation=True)

In [12]:
from transformers import DataCollatorWithPadding

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/venglov/.cache/huggingface/datasets/csv/default-2b849a0c54b184cb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-6e8c0cf78f86d373.arrow


In [13]:
tokenized_datasets = tokenized_datasets.remove_columns('opcodes')

In [14]:
from torch.utils.data import DataLoader

In [15]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)

In [16]:
# class SmartContractsDataset(tc.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#
#     def __len__(self):
#         return len(self.labels)
#
#     def __getitem__(self, idx):

#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['label'] = torch.tensor(self.labels[idx])
#         return item


In [17]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [18]:

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [19]:
# labels = (list(map(), labels)))

In [20]:
# labels = torch.Tensor(labels)

In [21]:
# from transformers import AdamW
#
# optimizer = AdamW(model.parameters(), lr=1e-5)


In [22]:
# from transformers import get_scheduler
#
# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=600,
#     num_training_steps=num_training_steps,
# )

In [23]:
device = "cuda"


In [24]:
model.to(device)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [25]:
# from tqdm.auto import tqdm
#
# progress_bar = tqdm(range(num_training_steps))
#
# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)


In [26]:
# scd = SmartContractsDataset(inputs, sc_df.malicious.tolist())
# train_size = int(0.8 * len(scd))
# val_size = int(0.1 * len(scd))
# test_size = len(scd) - train_size - val_size
# training_dataset, validation_dataset, testing_dataset = tc.utils.data.random_split(scd,
#                                                                                    [train_size, val_size, test_size])


In [27]:
# import evaluate
#
# metric = evaluate.load("precision")
#
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     # predictions = np.argmax(logits, axis=-1)
#     loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]))
#     loss = loss_fct(logits.view(-1, 2), labels.view(-1))
#     return loss

# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=3,
#     # use_mps_device=True,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=600,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     evaluation_strategy="epoch"
# )
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

In [28]:
from transformers import get_scheduler
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=600,
    num_training_steps=num_training_steps,
)

loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]).to(device))

In [29]:
torch.cuda.empty_cache()

In [30]:
from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    loss_total = 0
    loss_val = 0
    model.train()
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        # optimizer.zero_grad()
        outputs = model(**batch)
        # loss = outputs.loss

        logits = outputs.logits
        # predictions = torch.argmax(logits, dim=-1)
        loss = loss_fn(logits.view(-1, 2), batch["labels"].view(-1))
        # metric.add_batch(predictions=predictions, references=batch["labels"])

        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)
        loss_total += loss.item()

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        loss = loss_fn(logits.view(-1, 2), batch["labels"].view(-1))
        loss_val += loss.item()

    #
    #     logits = outputs.logits
    #     predictions = torch.argmax(logits, dim=-1)
    #     metric.add_batch(predictions=predictions, references=batch["labels"])
    # metric.compute()
    print(f"Epoch: {epoch + 1}/{num_epochs}; Training loss: {loss_total / len(train_dataloader)}; Validation loss: {loss_val / len(eval_dataloader)}")
    # print("")

100%|██████████| 579/579 [01:43<00:00,  5.62it/s]


Epoch: 1/3; Training loss: 0.10887776911031496; Validation loss: 0.03267311663904156


100%|██████████| 579/579 [01:43<00:00,  5.59it/s]


Epoch: 2/3; Training loss: 0.04100818262788964; Validation loss: 0.019243060688201623


100%|██████████| 579/579 [01:43<00:00,  5.59it/s]


Epoch: 3/3; Training loss: 0.030161021032363792; Validation loss: 0.013387001610737151


In [31]:
# trainer.train()

In [32]:
torch.save(model.state_dict(), '/home/venglov/Documents/mcd_data/data_v2/opcodes_model_weights_v6.pth')

In [33]:
results = []

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    print(logits)
    predictions = torch.argmax(logits, dim=-1)
    # print(predictions, batch['labels'])
    for index in range(len(predictions)):
        results.append((batch['labels'][index], predictions[index]))


tensor([[ 3.8802, -4.1542],
        [ 3.8802, -4.1542],
        [ 3.8802, -4.1542],
        [ 3.8201, -4.1016],
        [ 3.8174, -4.0716],
        [ 3.8327, -4.0929],
        [ 3.4179, -3.6724],
        [ 3.8679, -4.1362],
        [ 3.8361, -4.1096],
        [ 3.5788, -3.7746],
        [ 3.8126, -4.0762],
        [ 3.8802, -4.1542],
        [ 3.8460, -4.1076],
        [ 3.7817, -4.0397],
        [ 3.7998, -4.0606],
        [ 3.8335, -4.0700]], device='cuda:0')
tensor([[ 3.7839, -4.0929],
        [ 3.7030, -4.0203],
        [ 3.8802, -4.1542],
        [ 3.7983, -4.0893],
        [ 3.8466, -4.0999],
        [ 3.6843, -3.8951],
        [ 3.7403, -4.0671],
        [ 3.7430, -4.0643],
        [ 3.8693, -4.1375],
        [ 3.8256, -4.1071],
        [ 3.8256, -4.1071],
        [ 3.8463, -4.1188],
        [ 3.8056, -4.0891],
        [ 3.8256, -4.1071],
        [ 3.7359, -4.0366],
        [ 3.7506, -4.0738]], device='cuda:0')
tensor([[ 3.6335, -3.9235],
        [ 0.1453, -0.2005],
        [ 3.

In [34]:
# results = []
# model.to('cuda')
# for i in range(len(testing_dataset)):
#     code = testing_dataset[i]
#     label = code['label']
#     input_ids = torch.tensor(code["input_ids"]).to('cuda')
#     attention_mask = torch.tensor(code["attention_mask"]).to('cuda')
#     with torch.no_grad():
#         outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
#         print(outputs)
#     y = np.argmax(outputs[0].to('cpu').numpy())
#     results.append((label, y))

In [35]:
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

print(results[0])
# Generates count of inference quadrants
for real, predicted in results:
    if real == True and predicted == 1:
        true_positive += 1
    elif real == False and predicted == 0:
        true_negative += 1
    elif real == True and predicted == 0:
        false_negative += 1
    elif real == False and predicted == 1:
        false_positive += 1

# print(l2)
print("True Positive:", true_positive)
print("True Negative:", true_negative)
print("False Positive:", false_positive)
print("False Negative:", false_negative)

# Machine Learning statistics and visuals. https://towardsdatascience.com/accuracy-recall-precision-f-score-specificity-which-to-optimize-on-867d3f11124
print("Accuracy:", (true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative))
precision = true_positive/(true_positive + false_positive)
print("Precision:", precision)
recall = true_positive/(true_positive + false_negative)
print("Recall:", recall)
print("F1-score", 2*(recall * precision)/(recall + precision))
print("Specificity:", true_negative/(true_negative + false_positive))

(tensor(0, device='cuda:0'), tensor(0, device='cuda:0'))
True Positive: 9
True Negative: 1017
False Positive: 2
False Negative: 1
Accuracy: 0.9970845481049563
Precision: 0.8181818181818182
Recall: 0.9
F1-score 0.8571428571428572
Specificity: 0.9980372914622179


In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
real = []
predicted = []
for r, p in results:
    real.append(r)
    predicted.append(p)
cf_matrix = confusion_matrix(real, predicted)
df_cm = pd.DataFrame(cf_matrix, index=[0, 1],
                     columns=[0, 1])
plt.figure(figsize=(12, 7))
sn.heatmap(df_cm, annot=True)

In [ ]:
torch.save(model.state_dict(), '/home/venglov/Documents/mcd_data/data_v2/opcodes_model_weights_v2.pth')

In [ ]:
model.load_state_dict(torch.load("/home/venglov/Documents/mcd_data/data_v2/opcodes_model_weights_v2.pth"))

In [ ]:
model.eval()


In [ ]:
model.to(device)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [ ]:
w3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
address = "0x2c2c667a5794458219960a007b7ac2f670c35973"

code = w3.eth.get_code(Web3.to_checksum_address(address))

In [ ]:
code

In [ ]:
opcode = EvmBytecode(code).disassemble()
opcodes = ''
for j in opcode:
    opcodes += f'{j.name} '
    if j.operand:
        opcodes += f'0x{j.operand} '
opcodes = opcodes[:-1]

In [ ]:
opcodes

In [ ]:
tokenizer.tokenize(opcodes)

In [ ]:
inputs = tokenizer([opcodes], padding="max_length", truncation=True)


In [ ]:
inputs

In [ ]:
item = {key: torch.tensor(val[0]) for key, val in inputs.items()}

In [ ]:
input_ids = torch.tensor(item["input_ids"]).to('cpu')
model.to('cpu')
attention_mask = torch.tensor(item["attention_mask"]).to('cpu')
print(attention_mask.unsqueeze(0))
with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
y = np.argmax(outputs[0].to('cpu').numpy())

In [ ]:
outputs[0]

In [ ]:
y